# Problem

This is problem 84 of Project Euler (https://projecteuler.net/problem=84).

We're interested in calculating the probability $P(s)$ of occupying a given square $s$ on the Monopoly board.

# Solution

The idea is to construct a Markov chain. That is, we identify the possible states $\psi$ that uniquely specify a probability of transition $P(\psi' | \psi)$ to the state $\psi'$ from $\psi$.

Here the rules depend on two pieces of data: the current square $s$, and the number of consecutive doubles rolled so far $d$ (the third consecutive double sends you to jail). So the transition matrix is $T = P(s', d' | s, d)$. Suppose then we calculated $T$. Marginalising, we have that $P(s,d)$ satisfies $P(s', d') = \sum_{s, d} P(s', d' | s, d) P(s, d)$, in other words $P(s,d)$ is a null vector of the matrix $T - \delta_{s,s'} \delta_{d, d'}$. Solving for $P(s,d)$ can be done using standard linear algebra methods. Finally we obtain $P(s)$ by marginalising over $d$, $P(s) = \sum_{d=0,1,2} P(s,d)$.

# Implementation 

We will need linear algebra methods:

In [1]:
import numpy as np
from scipy.linalg import null_space

We start by defining the probability distribution for two dice:

In [2]:
def two_dice_dist(throw, sides=6):
    """ Probability of a given throw for two dice """
    if throw < sides+1 and throw >= 2:
        return (throw-1)/sides**2
    if throw >= sides+1 and throw <= 2*sides:
        return (2*sides+1-throw)/sides**2

def throw_double(throw, sides=6):
    """ Relative probability of rolling a double for a given throw of two dice """
    if throw % 2 != 0:
        return 0
    else:
        return 1/(two_dice_dist(throw, sides)*sides**2)

The core of this problem is defining a class Monopoly that calculates the probability transition between each state.

In [3]:
class Monopoly:
    """ Calculates the probabilities of each square on a Monopoly board. """
    def __init__(self, sides=6):
        self.sides = sides
        self.square =\
        ['GO','A1','CC1','A2','T1','R1','B1','CH1','B2','B3','JAIL','C1','U1',
         'C2','C3','R2','D1','CC2','D2','D3','FP','E1','CH2','E2','E3','R3',
         'F1','F2','U2','F3','G2J','G1','G2','CC3','G3','R4','CH3','H1','T2','H2']
        self.index = dict()
        for label, i in zip(self.square, range(len(self.square))):
            self.index[label] = i

        self.dice_probability = [
            (throw,
             two_dice_dist(throw, sides),
             throw_double(throw, sides)
            ) for throw in range(2,2*sides+1)]

        self.prob_matrix = np.zeros((120,120))

    def next_railway(self, square):
        """ Returns the next railway square from given square. """
        i = self.index[square]
        if i >= 5 and i < 15:
            return 'R2'
        if i >= 15 and i < 25:
            return 'R3'
        if i >= 25 and i < 35:
            return 'R4'
        else:
            return 'R1'

    def next_utility(self, square):
        """ Returns the next utility square from given square. """
        i = self.index[square]
        if i >= 12 and i < 28:
            return 'U2'
        else:
            return 'U1'

    def compute_transition_probability(self, square, nb_double, next_square,
            next_nb_double, prob):
        """ Adds to each entry of the dictionary outcomes the probability of
        landing any square given we've advanced to next_square.
        """

        # local shorthand for readability
        def add_probability(next_square, prob):
            # defines P(new state | current state)
            self.prob_matrix[next_nb_double*40 + self.index[next_square],
                nb_double*40 + self.index[square]] += prob

        # In each case, the total probability adds up to prob
        # First treat special cases
        if next_square == 'G2J':
            add_probability('JAIL', prob)
        elif next_square[:2] == 'CC':  # community chest
            add_probability(next_square, 7/8*prob)
            add_probability('GO', 1/16*prob)
            add_probability('JAIL', 1/16*prob)
        elif next_square[:2] == 'CH':  # chance card
            add_probability(next_square, 3/8*prob) # go to next square
            for label in ['GO','JAIL','C1','E3','H2','R1']:
                add_probability(label, 1/16*prob)
            add_probability(self.next_railway(next_square), 1/8*prob)
            add_probability(self.next_utility(next_square), 1/16*prob)
            # also a chance of going back 3 squares
            new_next_square = self.square[(self.index[next_square]-3) %
                    len(self.square)]
            self.compute_transition_probability(square, nb_double, new_next_square,
                    next_nb_double, prob/16)
        else:
            # otherwise, normal square
            add_probability(next_square, prob)

    def compute_transition_matrix(self):
        for square in self.index:
            # throw is the result of the roll, prob the probility for the outcome
            # prob_double is the relative probability of rolling a double
            for throw, prob, prob_double in self.dice_probability:
                next_square = self.square[(self.index[square] + throw) %
                        len(self.square)]
                for nb_double in range(3):
                    # if we roll a double
                    if nb_double == 2:  # rule for 3 doubles
                        # add_probability('JAIL', prob_double*prob)
                        self.prob_matrix[0*40 + self.index['JAIL'],
                            2*40 + self.index[square]] += prob_double*prob
                    else:
                        self.compute_transition_probability(square, nb_double,
                                next_square, nb_double+1, prob_double*prob)
                    # if we don't roll a double, reset to zero
                    self.compute_transition_probability(square, nb_double,
                        next_square, 0, (1-prob_double)*prob)

    def compute_probability(self):
        self.compute_transition_matrix()
        # the probability is the null vector of the matrix A
        A = self.prob_matrix - np.identity(len(self.prob_matrix))
        p = null_space(A).transpose()[0]
        p = p/sum(p)  # need to normalise, since any multiple of p is null
        # sum over values of doubles
        prob_vec = [(p[i] + p[40+i] + p[80+i], self.square[i]) for i in
                range(len(self.square))]
        prob_vec.sort(reverse=True)
        return prob_vec

The result using 6-sided dice:

In [4]:
m = Monopoly(6)
prob_vec = m.compute_probability()
print("List of probabilities")
for p, l in prob_vec[:5]:
    print(l, ': {:.3f}%'.format(100*p))

List of probabilities
JAIL : 6.242%
E3 : 3.184%
GO : 3.095%
D3 : 3.087%
R3 : 3.066%


Indeed this agrees with the odds given in the problem statement. For 4-sided dice, the probabilities would be instead

In [5]:
m = Monopoly(4)
prob_vec = m.compute_probability()
print("List of probabilities")
for p, l in prob_vec[:5]:
    print(l, ': {:.3f}%'.format(100*p))

List of probabilities
JAIL : 7.023%
R2 : 3.617%
E3 : 3.288%
D1 : 3.227%
R3 : 3.111%
